# COVID-19 Data Visualization

__Objective:__ Create an interactive dashboard to visualize the spread of COVID-19 across different countries. The data should include daily confirmed cases, recoveries, and deaths.

__Data:__ You can use the COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University.

Expectations:

- Use interactive plots to allow users to select and compare different countries.

- Display trends for confirmed cases, recoveries, and deaths over time.

- Include tools for zooming and panning.
 
__Hint: __Use Plotly for creating interactive plots and Dash for building the interactive dashboard.

In [1]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go

# Load the data from the provided CSV file 
covid_data = pd.read_csv('datasets/merged_covid19_data.csv', low_memory=False)           # Update the path to your local file

# Convert 'Date' column to datetime
covid_data['Date'] = pd.to_datetime(covid_data['Date'])

# Initialize the Dash app
app = Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("COVID-19 Dashboard"),
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country} for country in covid_data['Country/Region'].unique()],
        value=['Afghanistan'],
        multi=True
    ),
    dcc.Graph(id='covid-time-series'),
    dcc.Graph(id='covid-daily-stats')
])

# Define the callback to update the plots based on selected countries
@app.callback(
    [Output('covid-time-series', 'figure'),
     Output('covid-daily-stats', 'figure')],
    [Input('country-dropdown', 'value')]
)
def update_graph(selected_countries):
    if not selected_countries:
        return {}, {}

    filtered_data = covid_data[covid_data['Country/Region'].isin(selected_countries)]
    
    # Cumulative data over time
    fig_time_series = go.Figure()
    for country in selected_countries:
        country_data = filtered_data[filtered_data['Country/Region'] == country]
        fig_time_series.add_trace(go.Scatter(x=country_data['Date'], y=country_data['Confirmed'],
                                             mode='lines', name=f'{country} Confirmed'))
        fig_time_series.add_trace(go.Scatter(x=country_data['Date'], y=country_data['Deaths'],
                                             mode='lines', name=f'{country} Deaths'))
        fig_time_series.add_trace(go.Scatter(x=country_data['Date'], y=country_data['Recovered'],
                                             mode='lines', name=f'{country} Recovered'))
    fig_time_series.update_layout(title='Cumulative COVID-19 Cases Over Time',
                                  xaxis_title='Date',
                                  yaxis_title='Count',
                                  hovermode='x unified')

    # Daily new data
    fig_daily_stats = go.Figure()
    for country in selected_countries:
        country_data = filtered_data[filtered_data['Country/Region'] == country]
        fig_daily_stats.add_trace(go.Scatter(x=country_data['Date'], y=country_data['Daily Confirmed'],
                                             mode='lines', name=f'{country} Daily Confirmed'))
        fig_daily_stats.add_trace(go.Scatter(x=country_data['Date'], y=country_data['Daily Deaths'],
                                             mode='lines', name=f'{country} Daily Deaths'))
        fig_daily_stats.add_trace(go.Scatter(x=country_data['Date'], y=country_data['Daily Recovered'],
                                             mode='lines', name=f'{country} Daily Recovered'))
    fig_daily_stats.update_layout(title='Daily COVID-19 Cases, Deaths, and Recoveries',
                                  xaxis_title='Date',
                                  yaxis_title='Count',
                                  hovermode='x unified')
    
    # Enable zooming and panning
    fig_time_series.update_xaxes(rangeslider_visible=True)
    fig_daily_stats.update_xaxes(rangeslider_visible=True)

    return fig_time_series, fig_daily_stats

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
